In [1]:
import time
import requests
import numpy as np
import pandas as pd
import logging
import datetime
from binance.um_futures import UMFutures
from binance.websocket.um_futures.websocket_client import UMFuturesWebsocketClient
from binance.lib.utils import config_logging

# where API KEY stored.(USE YOUR OWN KEY)
from cert import binanceKey

# my own library
from backTester.BackTester import BackTester
from backTester.BackDataLoader import BackDataLoader
from conditionGenerator.ConditionGenerator import ConditionGenerator

# visualization
import matplotlib.pyplot as plt
import seaborn as sns

In [10]:
# 현재 시간
 = time.mktime(datetime.datetime.today().timetuple())
int(timestamp_now*1000)

1671620646000

In [ ]:
# data loading
client = UMFutures()
# backdata 가져오기 (간격 : 5m)
# 2022-08-21 18:30:00 ~ 2022-10-09 09:25:00
first_openTime = 1665156600000 # 2022, 10, 8, 0, 30
df1 = BackDataLoader().get_backdata(client, '1m', startTime=first_openTime, limit=1500)
df2 = BackDataLoader().get_backdata(client, '1m', startTime=first_openTime-5*1500*60*1000, limit=1500)
df3 = BackDataLoader().get_backdata(client, '1m', startTime=first_openTime-2*5*1500*60*1000, limit=1500)
df4 = BackDataLoader().get_backdata(client, '1m', startTime=first_openTime-3*5*1500*60*1000, limit=1500)
df5 = BackDataLoader().get_backdata(client, '1m', startTime=first_openTime-4*5*1500*60*1000, limit=1500)
df6 = BackDataLoader().get_backdata(client, '1m', startTime=first_openTime-5*5*1500*60*1000, limit=1500)
df7 = BackDataLoader().get_backdata(client, '1m', startTime=first_openTime-6*5*1500*60*1000, limit=1500)
df8 = BackDataLoader().get_backdata(client, '1m', startTime=first_openTime-7*5*1500*60*1000, limit=1500)
df9 = BackDataLoader().get_backdata(client, '1m', startTime=first_openTime-8*5*1500*60*1000, limit=1500)
df10 = BackDataLoader().get_backdata(client, '1m', startTime=first_openTime-9*5*1500*60*1000, limit=1500)

backdata = pd.concat([df10, df9, df8, df7, df6, df5, df4, df3, df2, df1], axis=0)
backdata.reset_index(inplace=True, drop=True)